In [20]:
import json
from urllib import request, parse


url = 'http://localhost:9000/api/parentdatasets'

u = request.urlopen(url)

resp = u.read()

data = json.loads(resp)

print(data)




{'dataSets': [{'name': 'mtngla'}, {'name': 'scratch'}]}


In [29]:
import requests
import json

headers={'Content-Type':'application/json'}

url = 'http://localhost:9000/env/setenvironment'

envData = { 'name': 'local', 'outputCdfPath': 'C:\\Earthwave\\TempCdfs\\' }
envJson = json.dumps(envData)

envresp = requests.post(url, data=envJson, headers=headers)

print( envresp.text )

geturl = 'http://localhost:9000/env/getenvironment'

envgetresp = requests.get(geturl, headers=headers)

print(envgetresp.text)




Environment Set [local]
{"name":"local","outputCdfPath":"C:\\Earthwave\\TempCdfs\\"}


In [22]:


import pandas
import json
from pandas.io.json import json_normalize

url = 'http://localhost:9000/api/gridcells'



parms = {'dbName':'mtngla'
        ,'year':2010
        , 'month':9}

jsonstr = json.dumps(parms)

resp = requests.post(url, data=jsonstr, headers=headers)

data = json.loads(resp.text)

df = json_normalize(data['catalogueElements'])

df.info()

print(df)





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 22 columns):
count           105 non-null int64
dsName          105 non-null object
gridCellMaxX    105 non-null int64
gridCellMaxY    105 non-null int64
gridCellMinX    105 non-null int64
gridCellMinY    105 non-null int64
gridCellSize    105 non-null int64
maxLat          105 non-null float64
maxLon          105 non-null float64
maxTime         105 non-null int64
maxX            105 non-null float64
maxY            105 non-null float64
minLat          105 non-null float64
minLon          105 non-null float64
minTime         105 non-null int64
minX            105 non-null float64
minY            105 non-null float64
month           105 non-null int64
projection      105 non-null object
qualityCount    105 non-null int64
shardName       105 non-null object
year            105 non-null int64
dtypes: float64(8), int64(11), object(3)
memory usage: 18.1+ KB
     count  dsName  gridCellMaxX  gridCell

In [23]:
totalPoints = df['count'].sum()
maxNumPoints = df['count'].max()

df['qualityPerc'] = (df['qualityCount'] / df['count'])
    
maxQualityPerc = df['qualityPerc'].max()*100    
    
print("TP: [%d] Max Points: [%d] Max QualityPerc [%d]." % (totalPoints, maxNumPoints, maxQualityPerc ) )


TP: [316488] Max Points: [46795] Max QualityPerc [100].


In [24]:
url = 'http://localhost:9000/api/boundingbox/mtngla/mtngla'

resp = requests.get(url,headers=headers)

print(resp.text)



{"gridCellMinX":-1900000,"gridCellMaxX":1100000,"gridCellMinY":-1100000,"gridCellMaxY":5600000,"minTime":1279356087000,"maxTime":1285386965000,"totalPoints":635961,"numberOfShards":256}


In [5]:
url = 'http://localhost:9000/api/boundingbox/mtngla/mtngla'

parms = {'minX':-1900000
        ,'maxX':1100000
        ,'minY':-1100000
        ,'maxY':5600000
        ,'minT':1279356087000
        ,'maxT':1285386965000 }

jsonstr = json.dumps(parms)

resp = requests.post(url, data=jsonstr, headers=headers)

data = json.loads(resp.text)

df = json_normalize(data['boxes'])

df.to_csv("catalogue.csv")


In [6]:
url = 'http://localhost:9000/api/shards/mtngla/mtngla'

parms =  {'minX':100000
        ,'maxX':200000
        ,'minY':600000
        ,'maxY':700000
        ,'minT':1279356087000
        ,'maxT':1285386965000 }

jsonstr = json.dumps(parms)

resp = requests.post(url, data=jsonstr, headers=headers)

print( resp.text )
data = json.loads(resp.text)

df = json_normalize(data['shards'])

df['shardName'].to_csv('out.csv')




{"shards":[{"shardName":"C:\\Earthwave\\/mtngla/swath/y2010/m9/cell_HMA_100000_600000/GridCell_0.nc","minX":100004.13991936622,"maxX":183875.1376974166,"minY":600116.4263281849,"maxY":699998.844395846,"minT":1285386965000,"maxT":1285386965000,"numberOfPoints":46795},{"shardName":"C:\\Earthwave\\/mtngla/swath/y2010/m7/cell_HMA_100000_600000/GridCell_0.nc","minX":100000.04272332246,"maxX":199793.25951205063,"minY":601073.2954021548,"maxY":699983.2344152909,"minT":1280385958000,"maxT":1280558610000,"numberOfPoints":57438}]}


In [7]:
url = 'http://localhost:9000/point/getgeojson/mtngla/mtngla'

parms = {'minX':100000
        ,'maxX':200000
        ,'minY':600000
        ,'maxY':700000
        ,'minT':1279356087000
        ,'maxT':1285386965000 }

jsonstr = json.dumps(parms)

print("Starting")
resp = requests.post(url, data=jsonstr, headers=headers)

f = open("demogeojson.json", "w")

f.write(resp.text)

f.close()

print("Complete")


Starting
Complete


In [30]:
url = 'http://localhost:9000/point/netcdffile/mtngla/mtngla'

parms = {'minX':-1900000
        ,'maxX':1100000
        ,'minY':-1100000
        ,'maxY':5600000
        ,'minT':1279356087000
        ,'maxT':1285386965000 }

jsonstr = json.dumps(parms)

print("Starting")
resp = requests.post(url, data=jsonstr, headers=headers)

print(resp.text)

Starting
C:\Earthwave\TempCdfs\data.nc
